In [ ]:
%%capture

import warnings

warnings.filterwarnings("ignore")

import altair as alt
import branca
import branca.colormap as cm

import calitp_data_analysis.magics
import geopandas as gpd
import google.auth
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from omegaconf import OmegaConf
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS
from shared_utils import webmap_utils

readable_dict = OmegaConf.load("readable2.yml")
credentials, project = google.auth.default()

import _operator_grain_route_summary
import _operator_grain_route_dir_visuals as _report_operator_visuals
import _operator_grain_scheduled_service as _scheduled_service_hour_visuals

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
def formatted(number):
    return "{:,}".format(number)

In [ ]:
analysis_name = "City of Santa Monica"

In [ ]:
#  %%capture_parameters
#  analysis_name

In [ ]:
# Set drop down menu to be on the upper right for the charts
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

In [ ]:
ROUTE_DIR_MONTH_FILE = GTFS_DATA_DICT.digest_tables.monthly_route_schedule_vp_report
OPERATOR_PROFILES = GTFS_DATA_DICT.digest_tables.operator_profiles_report
OPERATOR_ROUTE_MAP = GTFS_DATA_DICT.digest_tables.operator_routes_map_report
SCHEDULED_SERVICES = GTFS_DATA_DICT.digest_tables.scheduled_service_hours_report
ROUTE_DIR_QTR_FILE = GTFS_DATA_DICT.digest_tables.quarterly_route_schedule_vp_report

In [ ]:
route_dir_month_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{ROUTE_DIR_MONTH_FILE}.parquet",
    filters=[[("Analysis Name", "==", analysis_name)]],
).drop_duplicates()

In [ ]:
route_dir_qtr_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{ROUTE_DIR_QTR_FILE}.parquet",
    filters=[[("Analysis Name", "==", analysis_name)]],
)

In [ ]:
route_df = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE_MAP}.parquet",
    filters=[[("analysis_name", "==", analysis_name)]],
    storage_options={"token": credentials.token},
)

In [ ]:
# Add number column so we can plot this map using the webmap app
route_df = (route_df
                          .reset_index(drop=False)
                          .rename(columns={"index": "Number"})
                         )

In [ ]:
profile_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_PROFILES}.parquet",
    filters=[[("analysis_name", "==", analysis_name)]],
)

In [ ]:
service_hours_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{SCHEDULED_SERVICES}.parquet",
    filters=[[("Analysis Name", "==", analysis_name)]],
)

In [ ]:
analysis_name_edited = analysis_name.replace(" ","_").lower()
operator_webmap_file = f"{analysis_name_edited}_routes"

# {analysis_name}

## Operator Overview

In [ ]:
try:
    date = profile_df["service_date"].dt.to_pydatetime()[0]
except:
    pass

In [ ]:
try:
    service_area = formatted(int(profile_df.service_area_sq_miles.values[0]))
    service_pop = formatted(int(profile_df.service_area_pop.values[0]))
except:
    pass

In [ ]:
try:
    display(
        Markdown(
            f"""{analysis_name} is headquartered in <b>{profile_df.hq_city.values[0]}</b> in the Urbanized Area of <b>{profile_df.primary_uza_name.values[0]}</b>.<br>
            This operator provides <b>{service_area}</b> square miles of public transit service, which has a service population of <b>{service_pop}</b>.<br>
            This organization is a {profile_df.reporter_type.values[0]}.<br>
            <b>Data Source</b>: <a href="https://www.transit.dot.gov/ntd/data-product/2022-annual-database-agency-information">National Transit Database</a> Annual Agency Information.
            """
        )
    )
except:
    pass

### Route Typologies

In [ ]:
try:
    display(
        Markdown(
            f"""The following data presents an overview of GTFS characteristics using data from the most recent date of
      <b>{date.strftime("%B")} {date.year}</b>."""
        )
    )
except:
    pass

In [ ]:
try:
    n_routes = formatted(profile_df["operator_n_routes"].values[0])
    display(
        Markdown(
            f"""{analysis_name} runs <b>{n_routes}</b> unique routes. Below is the breakdown of the routes and routes can belong to one or more categories.<p>
            Route categories are determined using a approach that looks at GTFS trips data
        alongside National Association of City Transportation Officials (NACTO)'s
        <a href="https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-route-types/">Transit Route Types</a> 
        and <a href= "https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-frequency-volume/">Frequency and Volume</a>
        guides. Please see the <a href="https://github.com/cal-itp/data-analyses/blob/main/gtfs_digest/methodology.md">methodology docs</a> for more details on this approach.
        """
        )
    )
except:
    display(Markdown(f"""{analysis_name} doesn't have an operator profile."""))

In [ ]:
try:
    display(_operator_grain_route_summary.route_typology_chart(profile_df))
except:
    display(
        Markdown(f"""{analysis_name} doesn't have route typology data available.""")
    )

### Service Area

In [ ]:
color_map = cm.linear.Spectral_11.scale(0, len(route_df))

In [ ]:
try:
    operator_map = webmap_utils.set_state_export(
    route_df[["Route","geometry", "Number"]],
    subfolder = "operator_digest/",
    filename=operator_webmap_file,
    map_title=f"Transit Routes for {analysis_name}",
    cmap=color_map,
    color_col="Number",
    overwrite = True
    )
    webmap_utils.render_spa_link(operator_map["spa_link"], text=f"Routes for {analysis_name}")
    webmap_utils.display_spa_map(operator_map["spa_link"])
except:
    display(Markdown(f"""{analysis_name} doesn't have an route geographies."""))

In [ ]:
try:
    n_service_miles = formatted(int(profile_df.operator_route_length_miles.values[0]))
    n_stops = formatted(int(profile_df["operator_n_stops"].values[0]))
    n_arrivals = formatted(int(profile_df["operator_n_arrivals"].values[0]))

    display(
        Markdown(
            f"""{analysis_name} provides <b>{n_service_miles}</b> miles of public transit in {profile_df['counties_served'].values[0]} counties.
            {analysis_name} stops at <b>{n_stops}</b> unique stops, 
                totaling to <b>{n_arrivals}</b> arrivals.
        """
        )
    )

except:
    pass

In [ ]:
try:
    display(_operator_grain_route_summary.percentile_routes_chart(route_df))
except:
    pass

### Service Hours 

In [ ]:
try:
    display(
        _scheduled_service_hour_visuals.scheduled_service_hr_graph(
            service_hours_df, "Weekday", readable_dict.weekday_scheduled_service_graph
        )
    )
except:
    pass

In [ ]:
try:
    display(
        _scheduled_service_hour_visuals.scheduled_service_hr_graph(
            service_hours_df, "Saturday", readable_dict.saturday_scheduled_service_graph
        )
    )
except:
    pass

In [ ]:
try:
    display(
        _scheduled_service_hour_visuals.scheduled_service_hr_graph(
            service_hours_df, "Sunday", readable_dict.sunday_scheduled_service_graph
        )
    )
except:
    pass

## Detailed Route Overview

In [ ]:
try:
    display(
        _report_operator_visuals.route_filter(route_dir_qtr_df, route_dir_month_df)
    )
except:
    display(Markdown(f"""{analysis_name} only has schedule data."""))